<a href="https://colab.research.google.com/github/varenyaa/Peoplebox_assignemnt/blob/main/Peoplebox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Historical Data Transformation
Objective: Transform current employee data from a columnar format into a historical, row-based versioning format suitable for database storage using Python.**


In [ ]:
import pandas as pd

# Read input CSV into DataFrame
input_file = 'employee_data.csv'
df = pd.read_csv("/content/input.csv")

# Initialize empty list for transformed data
transformed_data = []

# Function to handle data transformation for compensation, reviews, and engagement
def transform_data(df, column_prefix):
    for index, row in df.iterrows():
        employee_code = row['Employee Code']
        manager_employee_code = row['Manager Employee Code']
        last_compensation = row[column_prefix + ' 1'] if not pd.isna(row[column_prefix + ' 1']) else None
        last_pay_raise_date = row[column_prefix + ' 1 date'] if not pd.isna(row[column_prefix + ' 1 date']) else None

        for i in range(1, 3):
            compensation = row[column_prefix + ' ' + str(i)] if not pd.isna(row[column_prefix + ' ' + str(i)]) else None
            effective_date = pd.to_datetime(row[column_prefix + ' ' + str(i) + ' date']) if not pd.isna(row[column_prefix + ' ' + str(i) + ' date']) else None

            # Handling compensation, engagement, and review score transitions
            transformed_data.append({
                'Employee Code': employee_code,
                'Manager Employee Code': manager_employee_code,
                'Last Compensation': last_compensation,
                'Compensation': compensation,
                'Last Pay Raise Date': last_pay_raise_date,
                'Variable Pay': None,  # Assuming no variable pay in this transformation
                'Tenure in Org': None,  # Assuming tenure is not needed in this transformation
                'Performance Rating': None,  # Assuming performance rating is not needed in this transformation
                'Engagement Score': None,  # Assuming engagement score is not needed in this transformation
                'Effective Date': effective_date,
                'End Date': None
            })

            last_compensation = compensation
            last_pay_raise_date = effective_date

# Transform compensation data
transform_data(df, 'Compensation')

# Transform review data
transform_data(df, 'Review')

# Transform engagement data
transform_data(df, 'Engagement')

# Create DataFrame from transformed data
output_df = pd.DataFrame(transformed_data)

# Sort output data
output_df.sort_values(by=['Employee Code', 'Effective Date'], inplace=True)

# Calculate End Date
output_df['End Date'] = output_df.groupby('Employee Code')['Effective Date'].shift(-1) - pd.Timedelta(days=1)
output_df['End Date'].fillna(pd.Timestamp('2100-01-01'), inplace=True)

# Write transformed data to CSV
output_file = 'transformed_employee_data.csv'
output_df.to_csv(output_file, index=False)

print("Transformation completed. Output file:", output_file)

Transformation completed. Output file: transformed_employee_data.csv


<ipython-input-14-768877fd8550>:50: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  output_df = pd.DataFrame(transformed_data)


In [ ]:
import pandas as pd

# Read the transformed data from the CSV file
output_file = 'transformed_employee_data.csv'
transformed_df = pd.read_csv("/content/transformed_employee_data.csv")

# Display the contents of the DataFrame
print(transformed_df)

    Employee Code  Manager Employee Code  Last Compensation  Compensation  \
0               1                    NaN                NaN           NaN   
1               1                    NaN                NaN           NaN   
2               1                    NaN                NaN           NaN   
3               1                    NaN                NaN           NaN   
4               1                    NaN                NaN           NaN   
5               1                    NaN                NaN           NaN   
6               2                    1.0                4.0           4.0   
7               2                    1.0                9.0           9.0   
8               2                    1.0            10000.0       10000.0   
9               2                    1.0                4.0           5.0   
10              2                    1.0                9.0           9.5   
11              2                    1.0            10000.0       20000.0   

**2. You have a data set which is a response to a recently run survey in a company on different themes or "Drivers".
As a data analyst, you have to analyse the data set to give useful slice & dice to the HR leader so that they can take actions**
**bold text**

In [1]:
import pandas as pd
import numpy as np

# Generate sample data for Employees table
np.random.seed(0)
employees_data = {
    'id': range(1, 101),
    'name': ['Employee' + str(i) for i in range(1, 101)],
    'department': np.random.choice(['Sales', 'HR', 'Marketing', 'Finance'], size=100),
    'location': np.random.choice(['New York', 'London', 'Paris', 'Tokyo'], size=100),
    'gender': np.random.choice(['Male', 'Female', 'Others'], size=100),
    'age': np.random.randint(20, 60, size=100),
    'manager_id': np.random.choice(range(1, 21), size=100)  # Assuming 20 managers
}

# Generate sample data for Responses table
responses_data = {
    'id': range(1, 101),
    'driver_name': np.random.choice(['Role Clarity', 'Career Growth', 'Policies', 'Work-Life Balance'], size=100),
    'score': np.random.randint(1, 6, size=100),
    'employee_id': np.random.choice(range(1, 101), size=100)
}

# Create DataFrames for Employees and Responses
employees_df = pd.DataFrame(employees_data)
responses_df = pd.DataFrame(responses_data)

In [2]:
employees_df.head()

,id,name,department,location,gender,age,manager_id
0,1,Employee1,Sales,London,Female,44,15
1,2,Employee2,Finance,Paris,Others,53,18
2,3,Employee3,HR,New York,Others,25,20
3,4,Employee4,Sales,Tokyo,Others,56,4
4,5,Employee5,Finance,New York,Female,20,10


In [4]:
overall_satisfaction = responses_df.groupby('driver_name')['score'].mean().reset_index()
highest_satisfaction = overall_satisfaction.sort_values(by='score', ascending=False).head(1)
lowest_satisfaction = overall_satisfaction.sort_values(by='score').head(1)

print("Driver with highest average score:")
print(highest_satisfaction)
print("\nDriver with lowest average score:")
print(lowest_satisfaction)

Driver with highest average score:
         driver_name     score
3  Work-Life Balance  3.366667

Driver with lowest average score:
    driver_name     score
2  Role Clarity  2.578947


In [5]:
segmented_data = responses_df.merge(employees_df, left_on='employee_id', right_on='id')
segmented_satisfaction = segmented_data.groupby(['department', 'location', 'gender', 'age'])['score'].mean().reset_index()

print("Segmented satisfaction analysis:")
print(segmented_satisfaction)

Segmented satisfaction analysis:
   department  location  gender  age  score
0     Finance    London  Female   49    4.0
1     Finance    London    Male   41    4.0
2     Finance    London    Male   48    2.0
3     Finance    London    Male   52    3.5
4     Finance  New York  Female   20    3.0
..        ...       ...     ...  ...    ...
59      Sales     Tokyo  Female   24    4.0
60      Sales     Tokyo    Male   43    5.0
61      Sales     Tokyo  Others   32    5.0
62      Sales     Tokyo  Others   39    3.0
63      Sales     Tokyo  Others   56    1.5

[64 rows x 5 columns]


In [6]:
managerial_data = responses_df.merge(employees_df, left_on='employee_id', right_on='id')
managerial_satisfaction = managerial_data.groupby('manager_id')['score'].mean().reset_index()

print("Managerial influence analysis:")
print(managerial_satisfaction)

Managerial influence analysis:
    manager_id     score
0            1  1.500000
1            2  3.000000
2            3  2.000000
3            4  2.833333
4            5  3.200000
5            6  3.666667
6            7  3.166667
7            8  3.285714
8            9  3.000000
9           10  3.428571
10          11  2.750000
11          12  4.500000
12          13  2.333333
13          14  4.000000
14          15  2.500000
15          16  4.333333
16          17  2.000000
17          18  2.700000
18          19  2.400000
19          20  2.666667


In [7]:
correlation_matrix = responses_df.pivot_table(index='employee_id', columns='driver_name', values='score')
correlation = correlation_matrix.corr()

print("Correlation matrix:")
print(correlation)

Correlation matrix:
driver_name        Career Growth  Policies  Role Clarity  Work-Life Balance
driver_name                                                                
Career Growth           1.000000  0.366508           NaN           0.123466
Policies                0.366508  1.000000      0.645497           0.739510
Role Clarity                 NaN  0.645497      1.000000           0.000000
Work-Life Balance       0.123466  0.739510      0.000000           1.000000
